# This script processes SYCL and CUDA results 

1. Go to SYCL-CUDA directory and compile all versions with:
```
make -j6
```
2. Run the benchmarking script
```
cd scripts
python run_sycl_cuda.py
```
3. The output that we will process is `sycl_cuda_<hostname>.txt` that should be already in the `Results` directory.



In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the file with baseline results exists
sycl_file = f'sycl_cuda_{hostname}.txt'
# if it is not in Results
if not os.path.exists(sycl_file):
    # is the file already in the directory?
    assert os.path.exists(os.path.join(hostname, sycl_file)), f'File {sycl_file} not found: something went wrong with the sycl_cuda benchmark.'
# if it is in Results
else:
    # copy the file to the directory
    assert os.system(f'mv {sycl_file} {hostname}/') == 0, f'Failed to move {sycl_file} to {hostname}/'
# rename sycl_file
sycl_file = os.path.join(hostname, sycl_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean
from common.utils import get_best_optimization

def tokenize(filename):
    experiment ={}

    with open(filename) as f:
        for line in f:
            tokens = line.split()
            if "Running:" in tokens:
                version=tokens[1].split("/")[2]
                input=tokens[2].split("/")[3][:-4]
                cloud = {
                    "INAER_2011_Alcoy_Core": "Alcoy",
                    "BABCOCK_2017_Arzua_3B": "Arzua",
                    "V21_group1_densified_point_cloud": "BrionF",
                    "V19_group1_densified_point_cloud": "BrionU",
                }[input]
                maxNum=int(tokens[3])
                if version not in experiment:
                    
                    experiment[version]={}
                if cloud not in experiment[version]:
                    experiment[version][cloud]={}
                experiment[version][cloud][maxNum]=[]
            if 'Construction' in tokens:
                experiment[version][cloud][maxNum].append(float(tokens[5]))
            if "Stage1" in tokens:
                experiment[version][cloud][maxNum].append(float(tokens[5]))
            if "Stage2" in tokens:
                experiment[version][cloud][maxNum].append(float(tokens[5]))
            if "Total" in tokens and "KERNEL" in tokens:
                experiment[version][cloud][maxNum].append(float(tokens[5]))
            if 'TIME' in tokens:
                experiment[version][cloud][maxNum].append(float(tokens[6]))

    return experiment

res=tokenize(sycl_file)
#print(res)

In [ ]:
from common.utils import tokenize_sycl_cpu

res_cpu = tokenize_sycl_cpu(sycl_file)

# get the best configuration and fix res, because "tokenize" always select the greater number of threads by default
for version in res:
    if version not in res_cpu:
        continue
    for cloud in res[version]:
        for maxNum in res[version][cloud]:
            # select the best case
            bestnth = min(res_cpu[version][cloud][maxNum], key=lambda x: res_cpu[version][cloud][maxNum][x][4])
            # fix res
            res[version][cloud][maxNum] = res_cpu[version][cloud][maxNum][bestnth]


# Memo and NoMemo versions with best max number for each time (tree, owm, and total)

In [ ]:
allmaxnums=[i for i in res['owm-sycl-cpu']['Alcoy']]
print(allmaxnums)

for ver in res:
    print("Version {}".format(ver))
    for cloud in res[ver]:
        treetimes=np.array([res[ver][cloud][i][0] if res[ver][cloud][i] else 1e9 for i in res[ver][cloud]])
        bestMNtree=np.argmin(treetimes)
        bestTreeTime=treetimes[bestMNtree]
        owmtimes=np.array([res[ver][cloud][i][3] if res[ver][cloud][i] else 1e9 for i in res[ver][cloud]])
        bestMNowm=np.argmin(owmtimes)
        bestowmTime=owmtimes[bestMNowm]
        totaltimes=np.array([res[ver][cloud][i][4] if res[ver][cloud][i] else 1e9 for i in res[ver][cloud]])
        bestMNtotal=np.argmin(totaltimes)
        besttotalTime=totaltimes[bestMNtotal]
        print("Cloud {} has TreeC. time={:.2f} (maxN={}), OWM time={:.2f} (maxN={}) and Total Time={:.2f} (maxN={}) in ms.".format(cloud,\
                                 bestTreeTime,allmaxnums[bestMNtree],bestowmTime,allmaxnums[bestMNowm],besttotalTime,allmaxnums[bestMNtotal]))



In [ ]:
""" Print the best results for CPU version
"""
for ver in res_cpu:
    print("Version {}".format(ver))
    for cloud in res_cpu[ver]:
        treetimes=np.array([res_cpu[ver][cloud][i][j][0] if res_cpu[ver][cloud][i] else 1e9 for i in res_cpu[ver][cloud] for j in res_cpu[ver][cloud][i]])
        bestMNtree=np.argmin(treetimes)
        bestTreeTime=treetimes[bestMNtree]
        owmtimes=np.array([res_cpu[ver][cloud][i][j][3] if res_cpu[ver][cloud][i] else 1e9 for i in res_cpu[ver][cloud] for j in res_cpu[ver][cloud][i]])
        bestMNowm=np.argmin(owmtimes)
        bestowmTime=owmtimes[bestMNowm]
        totaltimes=np.array([res_cpu[ver][cloud][i][j][4] if res_cpu[ver][cloud][i] else 1e9 for i in res_cpu[ver][cloud] for j in res_cpu[ver][cloud][i]])
        bestMNtotal=np.argmin(totaltimes)
        besttotalTime=totaltimes[bestMNtotal]
        print(f"Cloud {cloud} has TreeC. time={bestTreeTime:.2f}, OWM time={bestowmTime:.2f}, and Total time={besttotalTime:.2f} in ms.")



# Execution time of memo versions for all maxNumbers

In [ ]:
def plot_alltimes(results,version):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    #fig = plt.figure()
    labels=['OWM','Tree Const.','OWM+Tree C.']
    x=list(results[version]['Alcoy'].keys())
    xticks=range(1,len(x)+1)
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in zip(range(4),['Alcoy','Arzua','BrionF','BrionU']):
        axs[i].plot(np.array(xticks), np.array([results[version][name][i][3] for i in x]), marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(xticks), np.array([results[version][name][i][0] for i in x]), marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(np.array(xticks), np.array([results[version][name][i][4] for i in x]), marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('MaxNumber', fontsize=xlabelfs)
        axs[i].set_xticks(ticks=xticks[::3], labels=x[::3],fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()

    fig.suptitle(f'{version} sequential time (millisec.) @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (millisec.)', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    #axs[i].show()

if hostname == 'bombay':
    plot_alltimes(res,"owm-sycl-cpu")
else:
    plot_alltimes(res,"owm-sycl-cpu")
    plot_alltimes(res,"owm-sycl-igpu")
    plot_alltimes(res,"owm-sycl-dgpu")
    plot_alltimes(res,"owm-cuda")

# Compare CPU-optim 4 with all nomemo versions and MaxNumber=512

In [ ]:
import pandas as pd
df=pd.read_csv(f'{hostname}/All_Optimizations-{hostname}.csv',sep=';')
# get only the rows with "Baseline" or "MinRad" in the "Optimization" column
dfcrop = df[df['Optimization'].str.contains('Baseline|MinRad')].copy()
dfcrop.reset_index(drop=True, inplace=True)
dfcrop.insert(4,"Total",0)
dfcrop['Total']=dfcrop['TimeTree']+dfcrop['TimeOWM']
print(dfcrop)
base=dfcrop.loc[4:7,'TimeTree':'Total']
basea=np.array(base)*1000 #in ms
print(basea)

In [ ]:
mN=512
for ver in res:
    if "nomemo" not in ver.split("-"):
        continue
    print("Version {}".format(ver))
    for cloud, i in zip(res[ver],range(4)):
        treetime=res[ver][cloud][mN][0]
        treesp=basea[i,0]/treetime
        owmtime=res[ver][cloud][mN][3]
        owmsp=basea[i,1]/owmtime
        totaltime=res[ver][cloud][mN][4]
        totalsp=basea[i,2]/totaltime
        print("Cloud {} with maxN={} has TreeC. time={:.2f} ({:.2f}x), OWM time={:.2f} ({:.2f}x) and Total Time={:.2f} ({:.2f}x) in ms.".format(cloud, mN,\
                                    treetime, treesp, owmtime, owmsp, totaltime, totalsp))

In [ ]:
df=pd.read_csv(f'{hostname}/All_Optimizations-{hostname}.csv',sep=';')
# get Total times
df.insert(4,"Total",0)
df['Total']=df['TimeTree']+df['TimeOWM']
# get the best optimization
_,best_label = get_best_optimization(df)
print(f'Best optimization for C++: {best_label}')
# get only the rows with "Baseline" and best_label in the "Optimization" column
dfcrop = df.loc[df['Optimization'].isin(['Baseline', best_label])].copy()
dfcrop.reset_index(drop=True, inplace=True)
print(dfcrop)
base=dfcrop.loc[dfcrop['Optimization'] == best_label,'TimeTree':'TimeOWM']
basea=np.array(base)*1000 #in ms
print(basea)
mN=512
for ver in res:
    if "nomemo" not in ver.split("-"):
        continue
    #print("Version {}".format(ver))
    times=np.array([[res[ver][cloud][mN][0],res[ver][cloud][mN][3]] for cloud in res[ver]])
    #for cloud in res[ver]:
        #print("Cloud {} with maxN={} has TreeC. time={:.2f}, OWM time={:.2f} in ms.".format(cloud,\
        #                     mN,res[ver][cloud][mN][0],res[ver][cloud][mN][3]))
    basea=np.hstack((basea,times))
print(basea)


In [ ]:
"""
    Select the best version for each cloud and maxN instead of fixed maxN
"""
baseb=np.array(base)*1000 #in ms
print(baseb)
# iterate over the different implementations
for ver in res:
    # only consider the "nomemo" versions
    if "nomemo" not in ver.split("-"):
        continue
    times = []
    for cloud in res[ver]:
        # select the best maxN for each cloud
        bestmN = min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4])
        # get the tree and OWM times
        times.append([res[ver][cloud][bestmN][0], res[ver][cloud][bestmN][3]])
    baseb=np.hstack((baseb,times))
print(baseb)

In [ ]:
"""
    Select the best CPU version for each cloud and maxN instead of fixed maxN
"""
base_cpu=np.array(base)*1000 #in ms
print(base_cpu)
# get the different number of threads evaluated
vnth = list(res_cpu['owm-sycl-cpu']['Alcoy'][8].keys())
print(vnth)
# iterate over the different implementations
for ver in res_cpu:
    # only consider the "nomemo" versions
    if "nomemo" not in ver.split("-"):
        continue
    for nth in vnth:
        times = []
        for cloud in res_cpu[ver]:
            # select the best maxN for each cloud
            bestmN = min(res_cpu[ver][cloud], key=lambda x: res_cpu[ver][cloud][x][nth][4])
            # get the tree and OWM times
            times.append([res_cpu[ver][cloud][bestmN][nth][0], res_cpu[ver][cloud][bestmN][nth][3]])
        base_cpu=np.hstack((base_cpu,times))
print(base_cpu)

In [ ]:
import seaborn as sns
def plot_nomemo(npa, npb, check_list=None, fig_label=''):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']
    # separate the bars
    sepfactor = 9/16

    # only for the default case: o4, S-CPU. S-iGPU, S-dGPU, CUDA
    mydict = {'TBB base': [0,1], 'S-CPU': [2,3], 'S-iGPU': [4,5], 'S-dGPU': [6,7], 'CUDA': [8,9]}
    if check_list:
        # drop all the times that are not in the check_list
        npa = npa[:,[mydict[i] for i in check_list]].copy()
        npa = npa.reshape(4,len(check_list)*2)
        npb = npb[:,[mydict[i] for i in check_list]].copy()
        npb = npb.reshape(4,len(check_list)*2)

    # get a color palette
    palette = sns.color_palette("deep")

    #fig = plt.figure()
    phases=['OWM Trav.','Tree Const.']
    clouds=['Alcoy','Arzua','BrionF','BrionU']
    if check_list:
        versions = check_list 
    else:
        versions=['TBB base','S-CPU','S-iGPU','S-dGPU','CUDA']
    x=np.arange(1,len(versions)+1)
    width=0.35
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True)#, sharey=True)
    for i,name in zip(range(len(clouds)),clouds):
        b=np.zeros(len(versions))
        bb=np.zeros(len(versions))
        for j,z in zip(phases,reversed(range(len(phases)))): #reversed to have OWM first
            axs[i].bar(x-(width*sepfactor), npa[i][z::2], width, label=j, bottom=b, color=palette[z])
            b=b+npa[i][z::2]
            axs[i].bar(x+(width*sepfactor), npb[i][z::2], width, label=f'{j} best mN', bottom=bb, color=palette[z], hatch='//')
            bb=bb+npb[i][z::2]            

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Version', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=versions,fontsize=xticksfs,rotation = 45)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'Execution time (ms.) of each version @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Time in milliseconds', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)
    pp = PdfPages(f"{hostname}/Time-nomemo{fig_label}-maxnum512-{hostname}.pdf")
    pp.savefig(fig)
    pp.close()

if hostname == 'bombay':
    plot_nomemo(basea, baseb, ['TBB base','S-CPU'])
else:
    plot_nomemo(basea, baseb)
    plot_nomemo(basea, baseb, ['TBB base','S-dGPU','CUDA'], 'NVIDIA')

In [ ]:
import seaborn as sns
def plot_nomemo_cpu(npa, num_threads, fig_label=''):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']
    # separate the bars
    sepfactor = 9/16

    versions = ['TBB base']
    for nth in num_threads:
        # add the version for each number of threads
        versions.append(f'S-CPU-{nth}')
        
    # only for the default case
    # mydict = {'TBB base': [0,1], 'S-CPU-8': [2,3], 'S-CPU-16': [4,5], 'S-CPU-24': [6,7]}
    # npa = npa[:,[mydict[i] for i in versions]].copy()
    # npa = npa.reshape(4,len(versions)*2)

    # get a color palette
    palette = sns.color_palette("deep")

    #fig = plt.figure()
    phases=['OWM Trav.','Tree Const.']
    clouds=['Alcoy','Arzua','BrionF','BrionU']

    x=np.arange(1,len(versions)+1)
    width=0.35
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True)#, sharey=True)
    for i,name in zip(range(len(clouds)),clouds):
        b=np.zeros(len(versions))
        for j,z in zip(phases,reversed(range(len(phases)))): #reversed to have OWM first
            axs[i].bar(x, npa[i][z::2], width, label=j, bottom=b, color=palette[z])
            b=b+npa[i][z::2]    

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Version', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=versions,fontsize=xticksfs,rotation = 45)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'Execution time (ms.) of each version @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Time in milliseconds', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)
    # pp = PdfPages(f"{hostname}/Time-nomemo{fig_label}-maxnum512-{hostname}.pdf")
    # pp.savefig(fig)
    # pp.close()

plot_nomemo_cpu(base_cpu, vnth)

# Memo figure

In [ ]:
base=dfcrop.loc[dfcrop['Optimization'] == best_label, 'TimeTree':'TimeOWM']
basea=np.array(base)*1000 #in ms
print(basea)
mN=512
for ver in res:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    #print("Version {}".format(ver))
    times=np.array([[res[ver][cloud][mN][0],res[ver][cloud][mN][3]] for cloud in res[ver]])
    #for cloud in res[ver]:
        #print("Cloud {} with maxN={} has TreeC. time={:.2f}, OWM time={:.2f} in ms.".format(cloud,\
        #                     mN,res[ver][cloud][mN][0],res[ver][cloud][mN][3]))
    basea=np.hstack((basea,times))
print(basea)

In [ ]:
"""
    Select the best version for each cloud and maxN instead of fixed maxN
"""
baseb=np.array(base)*1000 #in ms
print(baseb)
# iterate over the different implementations
for ver in res:
    # only consider the "nomemo" versions
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    times = []
    for cloud in res[ver]:
        # select the best maxN for each cloud
        bestmN = min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4] if res[ver][cloud][x] else 1e9)
        # get the tree and OWM times
        times.append([res[ver][cloud][bestmN][0], res[ver][cloud][bestmN][3]])
    baseb=np.hstack((baseb,times))
print(baseb)

In [ ]:
"""
    Select the best CPU version for each cloud and maxN instead of fixed maxN
"""
base_cpu_mem=np.array(base)*1000 #in ms
print(base_cpu_mem)
# get the different number of threads evaluated
vnth = list(res_cpu['owm-sycl-cpu']['Alcoy'][8].keys())
print(vnth)
# iterate over the different implementations
for ver in res_cpu:
    # only consider the "nomemo" versions
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    for nth in vnth:
        times = []
        for cloud in res_cpu[ver]:
            # select the best maxN for each cloud
            bestmN = min(res_cpu[ver][cloud], key=lambda x: res_cpu[ver][cloud][x][nth][4])
            # get the tree and OWM times
            times.append([res_cpu[ver][cloud][bestmN][nth][0], res_cpu[ver][cloud][bestmN][nth][3]])
        base_cpu_mem=np.hstack((base_cpu_mem,times))
print(base_cpu_mem)

In [ ]:
if hostname == 'bombay':
    plot_nomemo(basea, baseb, ['TBB base','S-CPU'], 'memo')
else:
    plot_nomemo(basea, baseb, fig_label='memo')
    plot_nomemo(basea, baseb, ['TBB base','S-dGPU','CUDA'], 'memoNVIDIA')

In [ ]:
plot_nomemo_cpu(base_cpu_mem, vnth, fig_label='memo')

# OWM time Memo vs NoMemo for MaxNumber=512

In [ ]:
#memo vs no memo comparison
mN=512
for ver in res:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    print("Version {}".format(ver))
    for cloud in res[ver]:
        owmtimeNoMemo=res[f'{ver}-nomemo'][cloud][mN][3]
        owmtimeMemo=res[ver][cloud][mN][3]
        print("Cloud {} with maxN={} has OWM time={:.2f} ms  that is {:.2f}x faster than without memo (time={:.2f} ms)".format(cloud,mN, owmtimeMemo,\
                                                       owmtimeNoMemo/owmtimeMemo, owmtimeNoMemo ))

In [ ]:
print("\\begin{tabular}{|c|ccc|ccc|ccc|}\hline")
print(" & \multicolumn{3}{c|}{S-CPU}  & \multicolumn{3}{c|}{S-dGPU} & \multicolumn{3}{c|}{CUDA} \\\\ ")  
print(" Cloud &  NM &  M & IMP &  NM &  M & IMP &  NM &  M & IMP \\\\ \hline")
mN=512
versions=res.keys()
clouds=res['owm-sycl-cpu'].keys()
for i in clouds:
    print(i,end='')
    for j in versions:
        if "nomemo" in j.split("-") or "igpu" in j.split("-"):
            continue
        # exists the nomemo version for comparison?
        if f'{j}-nomemo' not in res:
            continue
        print("& {:0.2f} & {:0.2f} & {:0.2f}x ".format(res[j+"-nomemo"][i][mN][3],res[j][i][mN][3],res[j+"-nomemo"][i][mN][3]/res[j][i][mN][3]),end='')
    print("\\\\ \hline")
print("\\end{tabular}")

In [ ]:
"""
    Compare the memo vs no memo versions, but selecting the best maxNumber for each case
"""
for ver in res:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    print("Version {}".format(ver))
    for cloud in res[ver]:
        mN = min(res[f'{ver}-nomemo'][cloud], key=lambda x: res[f'{ver}-nomemo'][cloud][x][3])
        owmtimeNoMemo=res[f'{ver}-nomemo'][cloud][mN][3]
        mN = min(res[ver][cloud], key=lambda x: res[ver][cloud][x][3])
        owmtimeMemo=res[ver][cloud][mN][3]
        print("Cloud {} with maxN={} has OWM time={:.2f} ms  that is {:.2f}x faster than without memo (time={:.2f} ms)".format(cloud, mN, owmtimeMemo,\
                                                       owmtimeNoMemo/owmtimeMemo, owmtimeNoMemo ))

In [ ]:
"""
    Compare the memo vs no memo Total times, but selecting the best maxNumber for each case
"""
for ver in res:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    print("Version {}".format(ver))
    for cloud in res[ver]:
        mN = min(res[f'{ver}-nomemo'][cloud], key=lambda x: res[f'{ver}-nomemo'][cloud][x][4])
        totaltimeNoMemo=res[f'{ver}-nomemo'][cloud][mN][4]
        mN = min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4])
        totaltimeMemo=res[ver][cloud][mN][4]
        print("Cloud {} with maxN={} has OWM time={:.2f} ms  that is {:.2f}x faster than without memo (time={:.2f} ms)".format(cloud, mN, totaltimeMemo,\
                                                       totaltimeNoMemo/totaltimeMemo, totaltimeNoMemo ))

In [ ]:
"""
    Compare the memo vs no memo versions for CPU
"""
for ver in res_cpu:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    # let's consider different number of threads used as different versions
    for nth in vnth:
        print(f"Version {ver} {nth} threads")
        for cloud in res_cpu[ver]:
            mN = min(res_cpu[f'{ver}-nomemo'][cloud], key=lambda x: res_cpu[f'{ver}-nomemo'][cloud][x][nth][3])
            owmtimeNoMemo=res_cpu[f'{ver}-nomemo'][cloud][mN][nth][3]
            mN = min(res_cpu[ver][cloud], key=lambda x: res_cpu[ver][cloud][x][nth][3])
            owmtimeMemo=res_cpu[ver][cloud][mN][nth][3]
            print("Cloud {} {} threads with maxN={} has OWM time={:.2f} ms  that is {:.2f}x faster than without memo (time={:.2f} ms)".format(cloud, nth, mN, owmtimeMemo,\
                                                       owmtimeNoMemo/owmtimeMemo, owmtimeNoMemo ))

# Improvement factor when searching for the best maxNumber

In [ ]:
mN=512
for ver in res:
    if "nomemo" in ver.split("-"):
        continue
    # exists the nomemo version for comparison?
    if f'{ver}-nomemo' not in res:
        continue
    print("Version {}".format(ver))
    for cloud in res[ver]:
        bestMNtotal=min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4])
        bestTreeTime=res[ver][cloud][bestMNtotal][0]
        bestowmTime=res[ver][cloud][bestMNtotal][3]
        besttotalTime=res[ver][cloud][bestMNtotal][4]
        treetime=res[ver][cloud][mN][0]
        owmtime=res[ver][cloud][mN][3]
        totaltime=res[ver][cloud][mN][4]
        print("Cloud {} with MaxNum={} has TreeC. time={:.2f} ({:.2f}x), OWM time={:.2f} ({:.2f}x) and Total Time={:.2f} ({:.2f}x) in ms.".format(cloud,bestMNtotal,\
                                 bestTreeTime,treetime/bestTreeTime,bestowmTime,owmtime/bestowmTime,besttotalTime,totaltime/besttotalTime))

In [ ]:
print("\\begin{tabular}{|c|cccc|cccc|cccc|}\hline")
print(" & \multicolumn{4}{c|}{S-CPU} &  \multicolumn{4}{c|}{S-dGPU} & \multicolumn{4}{c|}{CUDA} \\\\ ")  
print(" Cloud &  MaxNum & Tree &  OWM & Tot &  MaxNum & Tree &  OWM & Tot &  MaxNum & Tree &  OWM & Tot  \\\\ \hline")
mN=512
versions=res.keys()
clouds=res['owm-sycl-cpu'].keys()
for cloud in clouds:
    print(cloud,end='')
    for ver in versions:
        if "nomemo" in ver.split("-") or "igpu" in ver.split("-"):
            continue
        # exists the nomemo version for comparison?
        if f'{ver}-nomemo' not in res:
            continue
        bestMNtotal=min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4])
        bestTreeTime=res[ver][cloud][bestMNtotal][0]
        bestowmTime=res[ver][cloud][bestMNtotal][3]
        besttotalTime=res[ver][cloud][bestMNtotal][4]
        treetime=res[ver][cloud][mN][0]
        owmtime=res[ver][cloud][mN][3]
        totaltime=res[ver][cloud][mN][4]
        print("& {} & {:0.2f}x & {:0.2f}x & {:0.2f}x ".format(bestMNtotal,treetime/bestTreeTime,owmtime/bestowmTime,totaltime/besttotalTime),end='')
    print("\\\\ \hline")
print("\\end{tabular}")

# Save results in All_OptimizationsSYCL-CUDA.csv

In [ ]:
output=f"{hostname}/All_OptimizationsSYCL-CUDA-{hostname}.csv"

f = open(output, "a")

print("Optimization;Cloud;TimeTree;TimeOWM;Level;MinRadMaxNum")
f.write("Optimization;Cloud;TimeTree;TimeOWM;Level;MinRadMaxNum\n")

for ver in res:
    #print("Version {}".format(ver))
    for cloud in res[ver]:
        totaltimes=np.array([res[ver][cloud][i][4] for i in res[ver][cloud] if res[ver][cloud][i]])
        bestMNtotal=min(res[ver][cloud], key=lambda x: res[ver][cloud][x][4] if res[ver][cloud][x] else 1e9)
        bestTreeTime=res[ver][cloud][bestMNtotal][0]
        bestowmTime=res[ver][cloud][bestMNtotal][3]
        print("{};{};{:.5f};{:.5f};{};{}".format(ver,cloud,bestTreeTime/1000,bestowmTime/1000,0,bestMNtotal))
        f.write("{};{};{:.5f};{:.5f};{};{}\n".format(ver,cloud,bestTreeTime/1000,bestowmTime/1000,0,bestMNtotal))
f.close()